# Data Visualization
By Intern: Diego Varela, Mentor: Surendra Adhikari

In [319]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import os
import pandas as pd
import datetime


In [ ]:
data_folder = '/Users/dlugardo/Desktop/data/ENU_v2' # path to the folder with the data 
metadata = 

def get_data(location):
    file_name = str(location) + '.ENU.txt'
    path = os.path.join(data_folder, file_name)

    if os.path.isfile(path):
        data = np.loadtxt(path, skiprows=2)
    else:
        file_name = str(location) + '_ENU.txt'
        path = os.path.join(data_folder, file_name)

        if os.path.isfile(path):
            data = np.loadtxt(path, skiprows=2)
        else:
            raise FileNotFoundError(f"Neither '{location}.ENU.txt' nor '{location}_ENU.txt' found in {data_folder}")
    return data

In [320]:
def decimal_year_to_date(decimal_year):
    """
    Converts a decimal year to a datetime.date object.
    """
    year = int(decimal_year)
    fractional_part = decimal_year - year

    # Determine if it's a leap year for accurate day calculation
    is_leap = (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0)
    days_in_year = 366 if is_leap else 365

    # Calculate the number of days from the start of the year
    days_offset = fractional_part * days_in_year

    # Create a datetime object for January 1st of that year
    start_of_year = datetime.date(year, 1, 1)

    # Add the calculated offset in days
    result_date = start_of_year + datetime.timedelta(days=days_offset)

    return result_date

# Example usage:
decimal_year_value = AASI_time[1261]
date_result = decimal_year_to_date(decimal_year_value)
#print(f"The decimal year {decimal_year_value} converts to: {date_result}")